# Практическое задание 1



## Замечания
* Задание необходимо сдать боту до 15.11.2021
* Соблюдаем кодекс чести (по нулям и списавшему, и давшему списать)
* Можно (и нужно!) применять для реализации только библиотеку **Numpy**
* Ничего, крому Numpy, нельзя использовать для реализации 
* **Keras** используется только для тестирования Вашей реализации
* Если какой-то из классов не проходит приведенные тесты, то соответствующее задание не оценивается
* Возможно использование дополнительных (приватных) тестов
 

In [1]:
# Вам понадобится для реализации
import keras.layers as layers
import numpy as np

# Нужно для тестирования
from tensorflow import keras

* Вспомогательные функции для тестирования

In [2]:
def compare_tensors(x, y, tol=0.001, test_name="Test"):
    assert x.shape == y.shape, test_name + " different shapes"
    diff = np.sum((y - x) ** 2)
    assert diff < tol, test_name + " Failed!"
    print(test_name + " Passed!")
    return

In [3]:
def compare_tensors_array(x, y, tol=0.001, test_name="Test"):
    assert len(x) == len(y), test_name + " different lengths"
    for i in range(len(x)):
        t = test_name + " subtest " + str(i)
        compare_tensors(x[i], y[i], tol=tol, test_name=t)
    print(test_name + " Passed!")
    return

* Шаблон класса любой операции (слоя), которую Вам необходимо будет реализовать

In [4]:
class Layer(object):
    def __init__(self):
        self.name = "Layer"

    def forward(self, input_data):
        pass

Рекомендуется запускать на GPU. В таком случае keras не ругается на channels_first



---



* (1 балл) Реализация "спрямляющего" слоя Flatten

In [5]:
class FlattenLayer(Layer):
    def __init__(self):
        self.name = "Flatten"

    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Преобразуем в двухмерный тензор: при этом по первой размерности НЕ преобразуем
        # Выкладываем данные: сначала по последней размерности, затем по предпоследней и т.д.
        # Нужно заполнить Numpy-тензор out
        data = np.swapaxes(input_data, 1, 3)
        out = np.reshape(np.swapaxes(data, 1, 2), (data.shape[0], -1))
        return out

    # решейп начинал расплющивать с последних размерностей, а флеттен от керас, наоборот, с внешних размерностей, поэтому пришлось
    # добавить свап осей

* Функция предварительного тестирования слоя **Flatten**
* Функции с названием "**test_**" не менять
* Вы можете самостоятельно поиграться с параметрами типа B/C/H/W etc

In [6]:
def test_FlattenLayer():
    B = 1
    C = 1
    H = 3
    W = 3
    x = np.random.randn(B, C, H, W)
    y = layers.Flatten(data_format="channels_first")
    y_keras = y(x).numpy()
    y_out = FlattenLayer().forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Flatten 1")
    B = 1
    C = 2
    H = 3
    W = 3
    x = np.random.randn(B, C, H, W)
    y = layers.Flatten(data_format="channels_first")
    y_keras = y(x).numpy()
    y_out = FlattenLayer().forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Flatten 2")
    return


def random_test_FlattenLayer(num_tests=10):
    for i in range(num_tests):
        B = np.random.randint(low=1, high=30)
        C = np.random.randint(low=1, high=30)
        H = np.random.randint(low=1, high=30)
        W = np.random.randint(low=1, high=30)
        print(f"Test {i+1}, B:{B}, C:{C}, H:{H}, W:{W}")
        x = np.random.randn(B, C, H, W)
        y = layers.Flatten(data_format="channels_first")
        y_keras = y(x).numpy()
        y_out = FlattenLayer().forward(x)
        compare_tensors(y_keras, y_out, tol=0.001, test_name=f"Test Flatten {i+1}")

In [7]:
test_FlattenLayer()

Test Flatten 1 Passed!
Test Flatten 2 Passed!


In [8]:
random_test_FlattenLayer()

Test 1, B:27, C:9, H:11, W:7
Test Flatten 1 Passed!
Test 2, B:28, C:24, H:3, W:15
Test Flatten 2 Passed!
Test 3, B:27, C:29, H:28, W:22
Test Flatten 3 Passed!
Test 4, B:3, C:12, H:14, W:5
Test Flatten 4 Passed!
Test 5, B:25, C:1, H:1, W:13
Test Flatten 5 Passed!
Test 6, B:5, C:1, H:8, W:1
Test Flatten 6 Passed!
Test 7, B:24, C:24, H:20, W:20
Test Flatten 7 Passed!
Test 8, B:13, C:17, H:1, W:20
Test Flatten 8 Passed!
Test 9, B:15, C:2, H:26, W:7
Test Flatten 9 Passed!
Test 10, B:20, C:25, H:15, W:18
Test Flatten 10 Passed!




---



* (1 балл) Реализация слоя субдискретизации **Global Average Pooling**

In [9]:
class GAP2DLayer(Layer):
    def __init__(self):
        self.name = "GAP2D"

    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Сворачиваем по двум последним размерностям (то есть на выходе - минус две размерности)
        # Нужно заполнить Numpy-тензор out
        return input_data.mean(axis=(2, 3))

In [10]:
def test_GAP2DLayer():
    B = 1
    C = 1
    H = 3
    W = 3
    x = np.random.randn(B, C, H, W)
    y = layers.GlobalAveragePooling2D(data_format="channels_first")
    y_keras = y(x).numpy()
    y_out = GAP2DLayer().forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test GAP2D 1")
    B = 1
    C = 2
    H = 3
    W = 3
    x = np.random.randn(B, C, H, W)
    y = layers.GlobalAveragePooling2D(data_format="channels_first")
    y_keras = y(x).numpy()
    y_out = GAP2DLayer().forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test GAP2D 2")
    return


def random_test_GAP2DLayer(num_tests=10):
    for i in range(num_tests):
        B = np.random.randint(low=1, high=30)
        C = np.random.randint(low=1, high=30)
        H = np.random.randint(low=1, high=30)
        W = np.random.randint(low=1, high=30)
        print(f"Test {i+1}, B:{B}, C:{C}, H:{H}, W:{W}")
        x = np.random.randn(B, C, H, W)
        y = layers.GlobalAveragePooling2D(data_format="channels_first")
        y_keras = y(x).numpy()
        y_out = GAP2DLayer().forward(x)
        compare_tensors(y_keras, y_out, tol=0.001, test_name=f"Test GAP2D {i+1}")

In [11]:
test_GAP2DLayer()

Test GAP2D 1 Passed!
Test GAP2D 2 Passed!


In [12]:
random_test_GAP2DLayer()

Test 1, B:25, C:15, H:3, W:16
Test GAP2D 1 Passed!
Test 2, B:16, C:18, H:12, W:23
Test GAP2D 2 Passed!
Test 3, B:4, C:15, H:2, W:19
Test GAP2D 3 Passed!
Test 4, B:25, C:1, H:20, W:20
Test GAP2D 4 Passed!
Test 5, B:3, C:13, H:22, W:16
Test GAP2D 5 Passed!
Test 6, B:15, C:14, H:11, W:5
Test GAP2D 6 Passed!
Test 7, B:24, C:27, H:27, W:28
Test GAP2D 7 Passed!
Test 8, B:2, C:15, H:17, W:17
Test GAP2D 8 Passed!
Test 9, B:16, C:28, H:15, W:24
Test GAP2D 9 Passed!
Test 10, B:1, C:17, H:27, W:6
Test GAP2D 10 Passed!




---



* (2 балла) Реализация слоя субдискретизации **MaxPooling**

In [13]:
class MaxPool2DLayer(Layer):
    def __init__(self, pool_size=2, stride=2):
        self.name = "MaxPool2D"
        self.pool_size = pool_size
        self.stride = stride

    def forward(self, input_data):
        out_h = int((input_data.shape[2] - (self.pool_size - 1) - 1) / self.stride + 1)
        out_w = int((input_data.shape[3] - (self.pool_size - 1) - 1) / self.stride + 1)
        out = np.empty((input_data.shape[0], input_data.shape[1], out_h, out_w))
        for i in range(out_h):
            for j in range(out_w):
                out[:, :, i, j] = np.max(
                    input_data[
                        :,
                        :,
                        i * self.stride : i * self.stride + self.pool_size,
                        j * self.stride : j * self.stride + self.pool_size,
                    ],
                    axis=(2, 3),
                )
        return out

In [14]:
def test_MaxPool2DLayer():
    B = 1
    C = 1
    H = 4
    W = 4
    pool_size = 2
    stride = 2
    x = np.random.randn(B, C, H, W)
    y = layers.MaxPooling2D(
        pool_size=pool_size,
        strides=stride,
        padding="valid",
        data_format="channels_first",
    )
    y_keras = y(x).numpy()
    y_out = MaxPool2DLayer(pool_size=pool_size, stride=stride).forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test MaxPool2D 1")
    B = 2
    C = 2
    H = 3
    W = 3
    pool_size = 2
    stride = 1
    x = np.random.randn(B, C, H, W)
    y = layers.MaxPooling2D(
        pool_size=pool_size,
        strides=stride,
        padding="valid",
        data_format="channels_first",
    )
    y_keras = y(x).numpy()
    y_out = MaxPool2DLayer(pool_size=pool_size, stride=stride).forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test MaxPool2D 2")
    return


def random_test_MaxPool2DLayer(num_tests=10):
    for i in range(num_tests):
        B = np.random.randint(low=1, high=30)
        C = np.random.randint(low=1, high=30)
        H = np.random.randint(low=2, high=30)
        W = np.random.randint(low=2, high=30)
        pool_size = np.random.randint(
            low=1, high=min(H, W)
        )  # т.к. нет паддинга, и не будем сворачивать больше самой картинки
        stride = np.random.randint(low=1, high=min(H, W))
        print(
            f"Test {i+1}, B:{B}, C:{C}, H:{H}, W:{W}, pool_size:{pool_size}, stride:{stride}"
        )
        x = np.random.randn(B, C, H, W)
        y = layers.MaxPooling2D(
            pool_size=pool_size,
            strides=stride,
            padding="valid",
            data_format="channels_first",
        )
        y_keras = y(x).numpy()
        y_out = MaxPool2DLayer(pool_size=pool_size, stride=stride).forward(x)
        compare_tensors(y_keras, y_out, tol=0.001, test_name=f"Test MaxPool2D {i+1}")

In [15]:
test_MaxPool2DLayer()

Test MaxPool2D 1 Passed!
Test MaxPool2D 2 Passed!


In [16]:
random_test_MaxPool2DLayer()

Test 1, B:17, C:3, H:20, W:8, pool_size:6, stride:2
Test MaxPool2D 1 Passed!
Test 2, B:9, C:11, H:22, W:6, pool_size:2, stride:4
Test MaxPool2D 2 Passed!
Test 3, B:5, C:25, H:6, W:20, pool_size:2, stride:5
Test MaxPool2D 3 Passed!
Test 4, B:2, C:1, H:24, W:14, pool_size:13, stride:7
Test MaxPool2D 4 Passed!
Test 5, B:29, C:28, H:14, W:11, pool_size:5, stride:7
Test MaxPool2D 5 Passed!
Test 6, B:3, C:23, H:25, W:8, pool_size:4, stride:5
Test MaxPool2D 6 Passed!
Test 7, B:2, C:14, H:17, W:23, pool_size:8, stride:11
Test MaxPool2D 7 Passed!
Test 8, B:9, C:11, H:26, W:25, pool_size:1, stride:17
Test MaxPool2D 8 Passed!
Test 9, B:5, C:13, H:6, W:14, pool_size:1, stride:2
Test MaxPool2D 9 Passed!
Test 10, B:14, C:14, H:10, W:19, pool_size:8, stride:3
Test MaxPool2D 10 Passed!




---



* (3 балла) Реализация слоя **активации** (поддерживаются **relu**, **sigmoid**, **softmax**)

In [17]:
class ActivationLayer(Layer):
    def __init__(self, activation="relu"):
        # Активация (поддерживаем 'relu', 'sigmoid', 'softmax')
        self.name = "Activation"
        self.activation = activation

    def forward(self, input_data):
        # На входе:
        # четырехмерный тензор вида [batch, input_channels, height, width] для 'relu', 'sigmoid'
        # или двухмерный тензор вида [batch, logits]
        # SoftMax применяется по последней размерности
        # Нужно заполнить Numpy-тензор out
        out = np.empty_like(input_data)
        if self.activation == "relu":
            input_data[input_data < 0] = 0
        elif self.activation == "sigmoid":
            input_data = 1 / (1 + np.exp(-input_data))
        elif self.activation == "softmax":
            for i in range(len(input_data)):
                denominator = sum(np.exp(input_data[i]))
                input_data[i] = np.exp(input_data[i]) / denominator

        return input_data

In [18]:
def test_ActivationLayer():
    B = 1
    C = 1
    H = 4
    W = 4
    activation = "relu"
    x = np.random.randn(B, C, H, W)
    y = layers.Activation(activation)
    y_keras = y(x).numpy()
    y_out = ActivationLayer(activation=activation).forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Activation 1")
    B = 2
    C = 2
    H = 3
    W = 3
    activation = "sigmoid"
    x = np.random.randn(B, C, H, W)
    y = layers.Activation(activation)
    y_keras = y(x).numpy()
    y_out = ActivationLayer(activation=activation).forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Activation 2")
    B = 3
    C = 10
    activation = "softmax"
    x = np.random.randn(B, C)
    y = layers.Activation(activation)
    y_keras = y(x).numpy()
    y_out = ActivationLayer(activation=activation).forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Activation 3")
    return


def random_test_ActivationLayer(num_tests=10):
    for i in range(num_tests):
        activations = ["relu", "sigmoid", "softmax"]
        activation = activations[np.random.randint(0, 3)]
        if activation == "softmax":
            B = np.random.randint(low=1, high=20)
            C = np.random.randint(low=1, high=20)
            print(f"Test {i+1}, B:{B}, C:{C}, activation: {activation}")
            x = np.random.randn(B, C)
            y = layers.Activation(activation)
            y_keras = y(x).numpy()
            y_out = ActivationLayer(activation=activation).forward(x)
            compare_tensors(
                y_keras, y_out, tol=0.001, test_name=f"Test Activation {i+1}"
            )
        else:
            B = np.random.randint(low=1, high=30)
            C = np.random.randint(low=1, high=30)
            H = np.random.randint(low=2, high=30)
            W = np.random.randint(low=2, high=30)
            print(f"Test {i+1}, B:{B}, C:{C}, H:{H}, W:{W}, activation: {activation}")
            x = np.random.randn(B, C, H, W)
            y = layers.Activation(activation)
            y_keras = y(x).numpy()
            y_out = ActivationLayer(activation=activation).forward(x)
            compare_tensors(
                y_keras, y_out, tol=0.001, test_name=f"Test Activation {i+1}"
            )

In [19]:
test_ActivationLayer()

Test Activation 1 Passed!
Test Activation 2 Passed!
Test Activation 3 Passed!


In [20]:
random_test_ActivationLayer()

Test 1, B:5, C:29, H:4, W:17, activation: sigmoid
Test Activation 1 Passed!
Test 2, B:4, C:1, activation: softmax
Test Activation 2 Passed!
Test 3, B:24, C:22, H:27, W:20, activation: relu
Test Activation 3 Passed!
Test 4, B:17, C:16, activation: softmax
Test Activation 4 Passed!
Test 5, B:13, C:23, H:24, W:19, activation: sigmoid
Test Activation 5 Passed!
Test 6, B:8, C:16, activation: softmax
Test Activation 6 Passed!
Test 7, B:13, C:5, H:29, W:16, activation: sigmoid
Test Activation 7 Passed!
Test 8, B:10, C:28, H:6, W:16, activation: sigmoid
Test Activation 8 Passed!
Test 9, B:9, C:8, activation: softmax
Test Activation 9 Passed!
Test 10, B:26, C:4, H:4, W:4, activation: relu
Test Activation 10 Passed!




---



* (3 балла) Реализация слоя пакетной нормализации **BatchNorm** (как для режима train, так и для режима test)

In [21]:
# Hint
# Train mode:
# out = (batch - mean(batch)) / sqrt(var(batch) + epsilon) * gamma + beta
# moving_mean = moving_mean * momentum + mean(batch) * (1 - momentum)
# moving_var = moving_var * momentum + var(batch) * (1 - momentum)
# Test mode:
# (batch - moving_mean) / sqrt(moving_var + epsilon) * gamma + beta


class BatchNormLayer(Layer):
    def __init__(
        self,
        momentum=0.99,
        epsilon=0.001,
        beta_init=None,
        gamma_init=None,
        moving_mean_init=None,
        moving_var_init=None,
        mode="train",
        input_channels=2,
    ):
        # mode: 'train', 'test'
        # Параметры gamma, beta, mean, var - все имеют размерность по количеству карт input_channels
        self.name = "BatchNorm"
        self.momentum = momentum
        self.epsilon = epsilon
        self.beta = beta_init
        self.gamma = gamma_init
        self.moving_mean = moving_mean_init
        self.moving_var = moving_var_init
        self.mode = mode
        self.input_channels = input_channels

    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # 1) Нужно заполнить Numpy-тензор out (той же размерности, что и вход)
        # 2) Нужно обновить moving_mean и moving_var в режиме 'train'
        out = np.empty_like(input_data)
        if self.mode == "train":
            for i in range(self.input_channels):
                mean = input_data[:, i, :, :].mean()
                var = input_data[:, i, :, :].var()
                out[:, i, :, :] = (input_data[:, i, :, :] - mean) / np.sqrt(
                    var + self.epsilon
                ) * self.gamma[i] + self.beta[i]
                self.moving_mean[i] = self.moving_mean[i] * self.momentum + mean * (
                    1 - self.momentum
                )
                self.moving_var[i] = self.moving_var[i] * self.momentum + var * (
                    1 - self.momentum
                )
        elif self.mode == "test":
            for i in range(self.input_channels):
                out[:, i, :, :] = (
                    (input_data[:, i, :, :] - self.moving_mean[i])
                    / np.sqrt(self.moving_var[i] + self.epsilon)
                    * self.gamma[i]
                    + self.beta[i]
                )

        return out

In [22]:
def test_BatchNormLayer():
    B = 2
    C = 2
    H = 4
    W = 4
    beta_init = 0 * np.ones(C)
    gamma_init = 1 * np.ones(C)
    moving_mean_init = 0 * np.ones(C)
    moving_var_init = 1 * np.ones(C)
    momentum = 0.99
    epsilon = 0.001
    mode = "train"
    x = np.random.randn(B, C, H, W)
    y = layers.BatchNormalization(
        axis=1, momentum=momentum, epsilon=epsilon, trainable=True
    )
    y_keras = y(x, training=True).numpy()
    y.set_weights([gamma_init, beta_init, moving_mean_init, moving_var_init])
    y_keras = y(x, training=True).numpy()
    y_out_layer = BatchNormLayer(
        momentum=momentum,
        epsilon=epsilon,
        beta_init=beta_init,
        gamma_init=gamma_init,
        moving_mean_init=moving_mean_init,
        moving_var_init=moving_var_init,
        mode=mode,
        input_channels=C,
    )
    y_out = y_out_layer.forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test BatchNorm 1")
    compare_tensors_array(
        y.get_weights(),
        [
            y_out_layer.gamma,
            y_out_layer.beta,
            y_out_layer.moving_mean,
            y_out_layer.moving_var,
        ],
        tol=0.00001,
        test_name="Test BatchNorm 1.1",
    )
    B = 2
    C = 2
    H = 4
    W = 4
    beta_init = 1 * np.ones(C)
    gamma_init = 0 * np.ones(C)
    moving_mean = 0 * np.ones(C)
    moving_var = 1 * np.ones(C)
    momentum = 0.99
    epsilon = 0.001
    mode = "test"
    x = np.random.randn(B, C, H, W)
    y = layers.BatchNormalization(
        axis=1, momentum=momentum, epsilon=epsilon, trainable=False
    )
    y_keras = y(x, training=False).numpy()
    y.set_weights([gamma_init, beta_init, moving_mean, moving_var])
    y_keras = y(x, training=False).numpy()
    y_out_layer = BatchNormLayer(
        momentum=momentum,
        epsilon=epsilon,
        beta_init=beta_init,
        gamma_init=gamma_init,
        moving_mean_init=moving_mean,
        moving_var_init=moving_var,
        mode=mode,
        input_channels=C,
    )
    y_out = y_out_layer.forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test BatchNorm 2")
    compare_tensors_array(
        y.get_weights(),
        [
            y_out_layer.gamma,
            y_out_layer.beta,
            y_out_layer.moving_mean,
            y_out_layer.moving_var,
        ],
        tol=0.00001,
        test_name="Test BatchNorm 2.1",
    )
    return


def random_test_BatchNormLayer(num_tests=10):
    for i in range(num_tests):
        B = np.random.randint(low=1, high=30)
        C = np.random.randint(low=1, high=30)
        H = np.random.randint(low=2, high=30)
        W = np.random.randint(low=2, high=30)
        beta_init = 1 * np.ones(C)
        gamma_init = 0 * np.ones(C)
        moving_mean = 0 * np.ones(C)
        moving_var = 1 * np.ones(C)
        momentum = np.random.uniform(low=0.0, high=1.0)
        print(f"Test {i+1}, B:{B}, C:{C}, H:{H}, W:{W}, momentum: {momentum}")
        epsilon = 0.001
        mode = "test"
        x = np.random.randn(B, C, H, W)
        y = layers.BatchNormalization(
            axis=1, momentum=momentum, epsilon=epsilon, trainable=False
        )
        y_keras = y(x, training=False).numpy()
        y.set_weights([gamma_init, beta_init, moving_mean, moving_var])
        y_keras = y(x, training=False).numpy()
        y_out_layer = BatchNormLayer(
            momentum=momentum,
            epsilon=epsilon,
            beta_init=beta_init,
            gamma_init=gamma_init,
            moving_mean_init=moving_mean,
            moving_var_init=moving_var,
            mode=mode,
            input_channels=C,
        )
        y_out = y_out_layer.forward(x)
        compare_tensors(y_keras, y_out, tol=0.001, test_name=f"Test BatchNorm {i+1}")
        compare_tensors_array(
            y.get_weights(),
            [
                y_out_layer.gamma,
                y_out_layer.beta,
                y_out_layer.moving_mean,
                y_out_layer.moving_var,
            ],
            tol=0.00001,
            test_name=f"Test BatchNorm {i+1}.1",
        )
    return

In [23]:
test_BatchNormLayer()

Test BatchNorm 1 Passed!
Test BatchNorm 1.1 subtest 0 Passed!
Test BatchNorm 1.1 subtest 1 Passed!
Test BatchNorm 1.1 subtest 2 Passed!
Test BatchNorm 1.1 subtest 3 Passed!
Test BatchNorm 1.1 Passed!
Test BatchNorm 2 Passed!
Test BatchNorm 2.1 subtest 0 Passed!
Test BatchNorm 2.1 subtest 1 Passed!
Test BatchNorm 2.1 subtest 2 Passed!
Test BatchNorm 2.1 subtest 3 Passed!
Test BatchNorm 2.1 Passed!


In [24]:
random_test_BatchNormLayer()

Test 1, B:20, C:2, H:29, W:27, momentum: 0.8504059464291097
Test BatchNorm 1 Passed!
Test BatchNorm 1.1 subtest 0 Passed!
Test BatchNorm 1.1 subtest 1 Passed!
Test BatchNorm 1.1 subtest 2 Passed!
Test BatchNorm 1.1 subtest 3 Passed!
Test BatchNorm 1.1 Passed!
Test 2, B:20, C:24, H:27, W:6, momentum: 0.8108210855484291
Test BatchNorm 2 Passed!
Test BatchNorm 2.1 subtest 0 Passed!
Test BatchNorm 2.1 subtest 1 Passed!
Test BatchNorm 2.1 subtest 2 Passed!
Test BatchNorm 2.1 subtest 3 Passed!
Test BatchNorm 2.1 Passed!
Test 3, B:3, C:27, H:6, W:22, momentum: 0.6747984657594498
Test BatchNorm 3 Passed!
Test BatchNorm 3.1 subtest 0 Passed!
Test BatchNorm 3.1 subtest 1 Passed!
Test BatchNorm 3.1 subtest 2 Passed!
Test BatchNorm 3.1 subtest 3 Passed!
Test BatchNorm 3.1 Passed!
Test 4, B:26, C:15, H:23, W:16, momentum: 0.6616104852309141
Test BatchNorm 4 Passed!
Test BatchNorm 4.1 subtest 0 Passed!
Test BatchNorm 4.1 subtest 1 Passed!
Test BatchNorm 4.1 subtest 2 Passed!
Test BatchNorm 4.1 subte



---



* (1 балл) Реализация **полносвязного** слоя

In [25]:
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, W_init=None, b_init=None):
        self.name = "Dense"
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.W = W_init
        self.b = b_init

    def forward(self, input_data):
        # На входе - двухмерный тензор вида [batch, input_channels]
        # Работаем по второй размерности, по первой размерности НЕ преобразуем
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out
        assert input_data.shape[1] == self.W.shape[0], "Mismatch in dimensions"
        out = input_data @ self.W + self.b
        return out

In [26]:
def test_DenseLayer():
    B = 1
    C_IN = 10
    C_OUT = 5
    x = np.random.randn(B, C_IN)
    W_init = np.random.randn(C_IN, C_OUT)
    b_init = np.random.randn(C_OUT)
    y = layers.Dense(C_OUT, use_bias=True)
    y_keras = y(x).numpy()
    y.set_weights([W_init, b_init])
    y_keras = y(x).numpy()
    y_out = DenseLayer(C_IN, C_OUT, W_init=W_init, b_init=b_init).forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Dense 1")
    B = 2
    C_IN = 5
    C_OUT = 10
    x = np.random.randn(B, C_IN)
    W_init = np.random.randn(C_IN, C_OUT)
    b_init = np.random.randn(C_OUT)
    y = layers.Dense(C_OUT, use_bias=True, input_shape=(C_IN,))
    y_keras = y(x).numpy()
    y.set_weights([W_init, b_init])
    y_keras = y(x).numpy()
    y_out = DenseLayer(C_IN, C_OUT, W_init=W_init, b_init=b_init).forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Dense 2")
    return


def random_test_DenseLayer(num_tests=10):
    for i in range(num_tests):
        B = np.random.randint(low=1, high=30)
        C_IN = np.random.randint(low=1, high=30)
        C_OUT = np.random.randint(low=1, high=30)
        print(f"Test {i+1}, B:{B}, C_IN:{C_IN}, C_OUT:{C_OUT}")
        x = np.random.randn(B, C_IN)
        W_init = np.random.randn(C_IN, C_OUT)
        b_init = np.random.randn(C_OUT)
        y = layers.Dense(C_OUT, use_bias=True)
        y_keras = y(x).numpy()
        y.set_weights([W_init, b_init])
        y_keras = y(x).numpy()
        y_out = DenseLayer(C_IN, C_OUT, W_init=W_init, b_init=b_init).forward(x)
        compare_tensors(y_keras, y_out, tol=0.001, test_name=f"Test Dense {i+1}")

In [27]:
test_DenseLayer()

Test Dense 1 Passed!
Test Dense 2 Passed!


In [28]:
random_test_DenseLayer()

Test 1, B:20, C_IN:14, C_OUT:9
Test Dense 1 Passed!
Test 2, B:19, C_IN:4, C_OUT:14
Test Dense 2 Passed!
Test 3, B:15, C_IN:2, C_OUT:1
Test Dense 3 Passed!
Test 4, B:9, C_IN:24, C_OUT:22
Test Dense 4 Passed!
Test 5, B:23, C_IN:16, C_OUT:25
Test Dense 5 Passed!
Test 6, B:28, C_IN:25, C_OUT:9
Test Dense 6 Passed!
Test 7, B:16, C_IN:25, C_OUT:25
Test Dense 7 Passed!
Test 8, B:28, C_IN:16, C_OUT:23
Test Dense 8 Passed!
Test 9, B:20, C_IN:20, C_OUT:19
Test Dense 9 Passed!
Test 10, B:12, C_IN:11, C_OUT:26
Test Dense 10 Passed!




---



* (2 балла) Реализация **сверточного** слоя

In [29]:
class Conv2DLayer(Layer):
    def __init__(
        self,
        kernel_size=3,
        input_channels=2,
        output_channels=3,
        padding="same",
        stride=1,
        K_init=None,
        b_init=None,
    ):
        # padding: 'same' или 'valid'
        # Работаем с квадратными ядрами, поэтому kernel_size - одно число
        # Работаем с единообразным сдвигом, поэтому stride - одно число
        # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
        self.name = "Conv2D"
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel = K_init
        self.bias = b_init
        self.padding = padding
        self.stride = stride

    def forward(self, input_data):
        if self.padding == "same":
            if input_data.shape[2] % self.stride == 0:
                pad_h = max(self.kernel_size - self.stride, 0)
            else:
                pad_h = max(self.kernel_size - (input_data.shape[2] % self.stride), 0)
            if input_data.shape[3] % self.stride == 0:
                pad_w = max(self.kernel_size - self.stride, 0)
            else:
                pad_w = max(self.kernel_size - (input_data.shape[3] % self.stride), 0)
            pad_top = pad_h // 2
            pad_bottom = pad_h - pad_top
            pad_left = pad_w // 2
            pad_right = pad_w - pad_left
            temp_input_data = np.pad(
                input_data,
                ((0, 0), (0, 0), (pad_top, pad_bottom), (pad_left, pad_right)),
            )
            temp_input_data[
                :,
                :,
                pad_top : pad_top + input_data.shape[2],
                pad_left : pad_left + input_data.shape[3],
            ] = input_data
        else:
            temp_input_data = input_data
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out

        out_h = (
            temp_input_data.shape[2] - self.kernel_size + self.stride
        ) // self.stride
        out_w = (
            temp_input_data.shape[3] - self.kernel_size + self.stride
        ) // self.stride
        out = np.zeros((input_data.shape[0], self.output_channels, out_h, out_w))
        for oc in range(self.output_channels):
            for i in range(out_h):
                for j in range(out_w):
                    out[:, oc, i, j] = (
                        np.tensordot(
                            temp_input_data[
                                :,
                                :,
                                i * self.stride : i * self.stride + self.kernel_size,
                                j * self.stride : j * self.stride + self.kernel_size,
                            ],
                            self.kernel[:, :, :, oc],
                            axes=([2, 3, 1], [0, 1, 2]),
                        )
                        + self.bias[oc]
                    )
        return out

In [30]:
def test_Conv2DLayer():
    B = 1
    C_IN = 1
    C_OUT = 1
    H = 10
    W = 10
    K = 3
    S = 1
    padding = "same"
    x = np.random.randn(B, C_IN, H, W)
    K_init = np.random.randn(K, K, C_IN, C_OUT)
    b_init = np.random.randn(C_OUT)
    y = layers.Conv2D(
        C_OUT,
        K,
        strides=S,
        padding=padding,
        data_format="channels_first",
        dilation_rate=1,
        groups=1,
        activation=None,
        use_bias=True,
    )
    y_keras = y(x).numpy()
    y.set_weights([K_init, b_init])
    y_keras = y(x).numpy()
    y_out = Conv2DLayer(
        kernel_size=K,
        input_channels=C_IN,
        output_channels=C_OUT,
        padding=padding,
        stride=S,
        K_init=K_init,
        b_init=b_init,
    ).forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Conv2D 1")
    B = 2
    C_IN = 3
    C_OUT = 5
    H = 9
    W = 9
    K = 3
    S = 2
    padding = "valid"
    x = np.random.randn(B, C_IN, H, W)
    K_init = np.random.randn(K, K, C_IN, C_OUT)
    b_init = np.random.randn(C_OUT)
    y = layers.Conv2D(
        C_OUT,
        K,
        strides=S,
        padding=padding,
        data_format="channels_first",
        dilation_rate=1,
        groups=1,
        activation=None,
        use_bias=True,
        input_shape=(H, W, C_IN),
    )
    y_keras = y(x).numpy()
    y.set_weights([K_init, b_init])
    y_keras = y(x).numpy()
    y_out = Conv2DLayer(
        kernel_size=K,
        input_channels=C_IN,
        output_channels=C_OUT,
        padding=padding,
        stride=S,
        K_init=K_init,
        b_init=b_init,
    ).forward(x)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Conv2D 2")
    return


def random_test_Conv2DLayer(num_tests=10):
    for i in range(num_tests):
        paddings = ["same", "valid"]
        padding = paddings[np.random.randint(low=0, high=2)]
        B = np.random.randint(low=1, high=30)
        C_IN = np.random.randint(low=1, high=30)
        C_OUT = np.random.randint(low=1, high=30)
        H = np.random.randint(low=2, high=30)
        W = np.random.randint(low=2, high=30)
        K = np.random.randint(
            low=1, high=30 if padding == "same" else max(2, min(H, W))
        )
        S = np.random.randint(
            low=1, high=30 if padding == "same" else max(2, min(H, W))
        )
        print(
            f"Test {i+1}, B:{B}, C_IN:{C_IN}, C_OUT:{C_OUT}, H:{H}, W:{W}, K:{K}, S:{S}, Padding:{padding}"
        )
        x = np.random.randn(B, C_IN, H, W)
        K_init = np.random.randn(K, K, C_IN, C_OUT)
        b_init = np.random.randn(C_OUT)
        y = layers.Conv2D(
            C_OUT,
            K,
            strides=S,
            padding=padding,
            data_format="channels_first",
            dilation_rate=1,
            groups=1,
            activation=None,
            use_bias=True,
        )
        y_keras = y(x).numpy()
        y.set_weights([K_init, b_init])
        y_keras = y(x).numpy()
        y_out = Conv2DLayer(
            kernel_size=K,
            input_channels=C_IN,
            output_channels=C_OUT,
            padding=padding,
            stride=S,
            K_init=K_init,
            b_init=b_init,
        ).forward(x)
        compare_tensors(
            y_keras, y_out, tol=0.001, test_name=f"Random test Conv2D {i+1}"
        )

In [31]:
random_test_Conv2DLayer()

Test 1, B:21, C_IN:28, C_OUT:5, H:16, W:28, K:15, S:2, Padding:valid
Random test Conv2D 1 Passed!
Test 2, B:13, C_IN:2, C_OUT:3, H:26, W:27, K:12, S:19, Padding:valid
Random test Conv2D 2 Passed!
Test 3, B:26, C_IN:5, C_OUT:29, H:26, W:24, K:29, S:26, Padding:same
Random test Conv2D 3 Passed!
Test 4, B:8, C_IN:15, C_OUT:14, H:29, W:2, K:1, S:1, Padding:valid
Random test Conv2D 4 Passed!
Test 5, B:10, C_IN:4, C_OUT:4, H:14, W:3, K:2, S:1, Padding:valid
Random test Conv2D 5 Passed!
Test 6, B:26, C_IN:27, C_OUT:16, H:11, W:19, K:4, S:10, Padding:valid
Random test Conv2D 6 Passed!
Test 7, B:17, C_IN:18, C_OUT:21, H:7, W:19, K:2, S:1, Padding:valid
Random test Conv2D 7 Passed!
Test 8, B:23, C_IN:5, C_OUT:13, H:10, W:17, K:1, S:1, Padding:valid
Random test Conv2D 8 Passed!
Test 9, B:9, C_IN:15, C_OUT:20, H:14, W:7, K:29, S:27, Padding:same
Random test Conv2D 9 Passed!
Test 10, B:26, C_IN:26, C_OUT:11, H:23, W:13, K:2, S:9, Padding:valid
Random test Conv2D 10 Passed!


In [32]:
test_Conv2DLayer()

Test Conv2D 1 Passed!
Test Conv2D 2 Passed!




---



* (2 балла) Реализация **транспонированного сверточного** слоя

In [33]:
class Conv2DTrLayer(Layer):
    def __init__(
        self,
        kernel_size=3,
        input_channels=2,
        output_channels=3,
        padding="valid",
        stride=1,
        K_init=None,
        b_init=None,
    ):
        # padding: число (сколько отрезать от модифицированной входной карты)
        # Работаем с квадратными ядрами, поэтому kernel_size - одно число
        # stride - одно число (коэффициент расширения)
        # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
        self.name = "Conv2DTr"
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel = K_init
        self.bias = b_init
        self.padding = padding
        self.stride = stride

    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out

        out_h = (input_data.shape[2] - 1) * self.stride + self.kernel_size
        out_w = (input_data.shape[3] - 1) * self.stride + self.kernel_size
        out = np.zeros((input_data.shape[0], self.output_channels, out_h, out_w))
        for oc in range(self.output_channels):
            for i in range(input_data.shape[2]):
                for j in range(input_data.shape[3]):
                    for k1 in range(self.kernel_size):
                        for k2 in range(self.kernel_size):
                            out[
                                :, oc, i * self.stride + k1, j * self.stride + k2
                            ] += np.sum(
                                self.kernel[k1, k2, oc, :] * input_data[:, :, i, j],
                                axis=1,
                            )
            out[:, oc] += self.bias[oc]

        p_left_top, p_right_bot = 0, 0
        if self.padding == "same":
            p_left_top = (self.kernel_size - self.stride) // 2
            p_right_bot = self.kernel_size - self.stride - p_left_top

        return out[
            :,
            :,
            p_left_top : out.shape[2] - p_right_bot,
            p_left_top : out.shape[3] - p_right_bot,
        ]

In [34]:
def adjust_kernel(K):
    K_new = K.copy()[::-1, ::-1, :, :]
    K_new = np.transpose(K_new, (0, 1, 3, 2))
    return K_new


def test_Conv2DTrLayer():
    B = 1
    C_IN = 1
    C_OUT = 1
    H = 3
    W = 3
    K = 3
    S = 2
    padding = 0
    x = np.random.randn(B, C_IN, H, W)
    K_init = np.random.randn(K, K, C_IN, C_OUT)
    b_init = np.random.randn(C_OUT)
    y = layers.Conv2DTranspose(
        C_OUT,
        K,
        strides=S,
        padding="valid",
        output_padding=None,
        data_format="channels_first",
        dilation_rate=1,
        groups=1,
        activation=None,
        use_bias=True,
    )
    y_keras = y(x).numpy()
    y.set_weights([adjust_kernel(K_init), b_init])
    y_keras = y(x).numpy()
    y_out = Conv2DTrLayer(
        kernel_size=K,
        input_channels=C_IN,
        output_channels=C_OUT,
        padding="valid",
        stride=S,
        K_init=adjust_kernel(K_init),
        b_init=b_init,
    ).forward(x)
    np.set_printoptions(precision=1)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Conv2DTr 1")

    B = 4
    C_IN = 2
    C_OUT = 3
    H = 3
    W = 3
    K = 3
    S = 2
    padding = "valid"
    x = np.random.randn(B, C_IN, H, W)
    K_init = np.random.randn(K, K, C_IN, C_OUT)
    b_init = np.random.randn(C_OUT)
    y = layers.Conv2DTranspose(
        C_OUT,
        K,
        strides=S,
        padding="valid",
        output_padding=None,
        data_format="channels_first",
        dilation_rate=1,
        groups=1,
        activation=None,
        use_bias=True,
    )
    y_keras = y(x).numpy()
    y.set_weights([adjust_kernel(K_init), b_init])
    y_keras = y(x).numpy()
    y_out = Conv2DTrLayer(
        kernel_size=K,
        input_channels=C_IN,
        output_channels=C_OUT,
        padding="valid",
        stride=S,
        K_init=adjust_kernel(K_init),
        b_init=b_init,
    ).forward(x)
    np.set_printoptions(precision=1)
    compare_tensors(y_keras, y_out, tol=0.001, test_name="Test Conv2DTr 2")
    return


def random_test_Conv2DTrLayer(num_tests=10):
    for i in range(num_tests):
        paddings = ["same", "valid"]
        padding = paddings[np.random.randint(low=0, high=2)]
        B = np.random.randint(low=1, high=30)
        C_IN = np.random.randint(low=1, high=30)
        C_OUT = np.random.randint(low=1, high=30)
        H = np.random.randint(low=2, high=30)
        W = np.random.randint(low=2, high=30)
        K = np.random.randint(low=1, high=30)
        S = np.random.randint(low=1, high=max(2, K))
        print(
            f"Test {i+1}, B:{B}, C_IN:{C_IN}, C_OUT:{C_OUT}, H:{H}, W:{W}, K:{K}, S:{S}, Padding:{padding}"
        )
        x = np.random.randn(B, C_IN, H, W)
        K_init = np.random.randn(K, K, C_IN, C_OUT)
        b_init = np.random.randn(C_OUT)
        y = layers.Conv2DTranspose(
            C_OUT,
            K,
            strides=S,
            padding=padding,
            output_padding=None,
            data_format="channels_first",
            dilation_rate=1,
            groups=1,
            activation=None,
            use_bias=True,
        )
        y_keras = y(x).numpy()
        y.set_weights([adjust_kernel(K_init), b_init])
        y_keras = y(x).numpy()
        y_out = Conv2DTrLayer(
            kernel_size=K,
            input_channels=C_IN,
            output_channels=C_OUT,
            padding=padding,
            stride=S,
            K_init=adjust_kernel(K_init),
            b_init=b_init,
        ).forward(x)
        np.set_printoptions(precision=1)
        compare_tensors(y_keras, y_out, tol=0.001, test_name=f"Test Conv2DTr {i+1}")

In [35]:
test_Conv2DTrLayer()

Test Conv2DTr 1 Passed!
Test Conv2DTr 2 Passed!


In [36]:
random_test_Conv2DTrLayer()

Test 1, B:16, C_IN:2, C_OUT:1, H:19, W:28, K:25, S:7, Padding:valid
Test Conv2DTr 1 Passed!
Test 2, B:26, C_IN:17, C_OUT:6, H:6, W:19, K:12, S:7, Padding:valid
Test Conv2DTr 2 Passed!
Test 3, B:21, C_IN:5, C_OUT:7, H:18, W:26, K:15, S:12, Padding:valid
Test Conv2DTr 3 Passed!
Test 4, B:15, C_IN:26, C_OUT:24, H:4, W:8, K:13, S:2, Padding:same
Test Conv2DTr 4 Passed!
Test 5, B:9, C_IN:12, C_OUT:3, H:28, W:4, K:20, S:7, Padding:valid
Test Conv2DTr 5 Passed!
Test 6, B:18, C_IN:5, C_OUT:6, H:7, W:7, K:12, S:8, Padding:same
Test Conv2DTr 6 Passed!
Test 7, B:28, C_IN:8, C_OUT:11, H:5, W:28, K:9, S:2, Padding:same
Test Conv2DTr 7 Passed!
Test 8, B:8, C_IN:27, C_OUT:8, H:9, W:11, K:18, S:1, Padding:same
Test Conv2DTr 8 Passed!
Test 9, B:17, C_IN:12, C_OUT:15, H:24, W:14, K:3, S:2, Padding:same
Test Conv2DTr 9 Passed!
Test 10, B:12, C_IN:22, C_OUT:13, H:9, W:2, K:26, S:15, Padding:same
Test Conv2DTr 10 Passed!
